<a href="https://colab.research.google.com/github/pk18singh/Flat_Prediction_Model/blob/main/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('properties_cleaned_v1.csv')

In [ ]:
df.duplicated().sum()

122

In [ ]:
df.head(1)
# focus is on -> areaWithType, additionalRoom, agePossession, furnishDetails, features

property_type      society     sector  price  price_per_sqft    area  \
0         house  independent  sector 81   0.89          8127.0  1095.0   

                         areaWithType  bedRoom  bathroom balcony  \
0  Built Up area: 1095 (101.73 sq.m.)        1         1       0   

  additionalRoom  floorNum facing agePossession  \
0  not available       1.0    NaN     undefined   

                                                                                                                                                                                                                           nearbyLocations  \
0  ['Orris Community Center', 'Petrol Pump', 'HP Petrol Pump', 'Essar Petrol Pump', 'Vijay Petrol Pump', 'Petrol Pump Indian Oil', 'Petrol pump Maitri motors', 'Indian Oil', "McDonald's", 'Rao Dhaba', 'KFC', 'Pizza Hut', "McDonald's"]   

                                                                                                                                                                                                                                                         furnishDetails  \
0  ['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']   

  features  
0      NaN

**1. areaWithType**

In [ ]:
df.sample(5)[['price','area','areaWithType']]

price    area  \
597    5.00  4300.0   
1118   1.15  1534.0   
1091   1.20  1350.0   
366    2.30  2585.0   
138    1.49  1963.0   

                                                                       areaWithType  
597                                                Carpet area: 4300 (399.48 sq.m.)  
1118   Super Built up area 1534(142.51 sq.m.)Carpet area: 1056 sq.ft. (98.11 sq.m.)  
1091                                         Super Built up area 1350(125.42 sq.m.)  
366                                          Super Built up area 2585(240.15 sq.m.)  
138   Super Built up area 1963(182.37 sq.m.)Carpet area: 1700 sq.ft. (157.94 sq.m.)

In [ ]:
# This function extracts the Super Built up area
def get_super_built_up_area(text):
    match = re.search(r'Super Built up area (\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [ ]:
# This function extracts the Built Up area or Carpet area
def get_area(text, area_type):
    match = re.search(area_type + r'\s*:\s*(\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [ ]:
# This function checks if the area is provided in sq.m. and converts it to sqft if needed
def convert_to_sqft(text, area_value):
    if area_value is None:
        return None
    match = re.search(r'{} \((\d+\.?\d*) sq.m.\)'.format(area_value), text)
    if match:
        sq_m_value = float(match.group(1))
        return sq_m_value * 10.7639  # conversion factor from sq.m. to sqft
    return area_value

In [ ]:
# Extract Super Built up area and convert to sqft if needed
df['super_built_up_area'] = df['areaWithType'].apply(get_super_built_up_area)
df['super_built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['super_built_up_area']), axis=1)

# Extract Built Up area and convert to sqft if needed
df['built_up_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Built Up area'))
df['built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['built_up_area']), axis=1)

# Extract Carpet area and convert to sqft if needed
df['carpet_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Carpet area'))
df['carpet_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['carpet_area']), axis=1)

In [ ]:
df[['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].sample(5)

price property_type    area  \
3643   1.95         house  1440.0   
3239   2.50          flat  1884.0   
1589   2.35          flat  1650.0   
8      0.85         house  1743.0   
38     1.01          flat  1750.0   

                                                                                                                areaWithType  \
3643                                                                                             Plot area 160(133.78 sq.m.)   
3239                                                                                        Carpet area: 1884 (175.03 sq.m.)   
1589                                                                                  Super Built up area 1650(153.29 sq.m.)   
8                                                                                         Built Up area: 1743 (161.93 sq.m.)   
38    Super Built up area 1750(162.58 sq.m.)Built Up area: 1475 sq.ft. (137.03 sq.m.)Carpet area: 1350 sq.ft. (125.42 sq.m.)   

      super_built_up_area  built_up_area  carpet_area  
3643                  NaN            NaN          NaN  
3239                  NaN            NaN       1884.0  
1589               1650.0            NaN          NaN  
8                     NaN         1743.0          NaN  
38                 1750.0         1475.0       1350.0

In [ ]:
df.duplicated().sum()

122

In [ ]:
df[~((df['super_built_up_area'].isnull()) | (df['built_up_area'].isnull()) | (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].shape

(534, 7)

In [ ]:
df[df['areaWithType'].str.contains('Plot')][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].head(5)

price property_type    area  \
4    2.00         house  1337.0   
10   3.80         house  1458.0   
11   0.70         house   864.0   
22   8.49         house  2700.0   
39   2.25         house   900.0   

                                                                                                     areaWithType  \
4                                                                                  Plot area 1337.4(124.25 sq.m.)   
10  Plot area 162(135.45 sq.m.)Built Up area: 162 sq.yards (135.45 sq.m.)Carpet area: 160 sq.yards (133.78 sq.m.)   
11                                                                                     Plot area 864(80.27 sq.m.)   
22                                                                                    Plot area 300(250.84 sq.m.)   
39                                                                                     Plot area 100(83.61 sq.m.)   

    super_built_up_area  built_up_area  carpet_area  
4                   NaN            NaN          NaN  
10                  NaN          162.0        160.0  
11                  NaN            NaN          NaN  
22                  NaN            NaN          NaN  
39                  NaN            NaN          NaN

In [ ]:
df.isnull().sum()

property_type             0
society                   1
sector                    0
price                    18
price_per_sqft           18
area                     18
areaWithType              0
bedRoom                   0
bathroom                  0
balcony                   0
additionalRoom            0
floorNum                 19
facing                 1105
agePossession             1
nearbyLocations         177
furnishDetails          981
features                635
super_built_up_area    1888
built_up_area          2616
carpet_area            1859
dtype: int64

In [ ]:
all_nan_df = df[((df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']]

In [ ]:
all_nan_df.head()

price property_type    area                    areaWithType  \
4    2.00         house  1337.0  Plot area 1337.4(124.25 sq.m.)   
11   0.70         house   864.0      Plot area 864(80.27 sq.m.)   
22   8.49         house  2700.0     Plot area 300(250.84 sq.m.)   
39   2.25         house   900.0      Plot area 100(83.61 sq.m.)   
45  13.50         house  4518.0     Plot area 502(419.74 sq.m.)   

    super_built_up_area  built_up_area  carpet_area  
4                   NaN            NaN          NaN  
11                  NaN            NaN          NaN  
22                  NaN            NaN          NaN  
39                  NaN            NaN          NaN  
45                  NaN            NaN          NaN

In [ ]:
all_nan_index = df[((df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].index

In [ ]:
# Function to extract plot area from 'areaWithType' column
def extract_plot_area(area_with_type):
    match = re.search(r'Plot area (\d+\.?\d*)', area_with_type)
    return float(match.group(1)) if match else None

In [ ]:
all_nan_df['built_up_area'] = all_nan_df['areaWithType'].apply(extract_plot_area)

# Update the original dataframe
#gurgaon_properties.update(filtered_rows)

In [ ]:
all_nan_df

price property_type      area                     areaWithType  \
4      2.00         house    1337.0   Plot area 1337.4(124.25 sq.m.)   
11     0.70         house     864.0       Plot area 864(80.27 sq.m.)   
22     8.49         house    2700.0      Plot area 300(250.84 sq.m.)   
39     2.25         house     900.0       Plot area 100(83.61 sq.m.)   
45    13.50         house    4518.0      Plot area 502(419.74 sq.m.)   
52    12.00         house    3150.0      Plot area 350(292.64 sq.m.)   
55     7.80         house    3370.0      Plot area 360(301.01 sq.m.)   
56    10.50         house    3240.0      Plot area 360(301.01 sq.m.)   
57    17.00         house    4500.0     Plot area 4500(418.06 sq.m.)   
70     0.27         house     225.0        Plot area 225(20.9 sq.m.)   
71     4.12         house    4635.0      Plot area 515(430.61 sq.m.)   
93     0.53         house     363.0       Plot area 360(33.45 sq.m.)   
94     0.82         house     900.0       Plot area 100(83.61 sq.m.)   
98     0.50         house     450.0       Plot area 450(41.81 sq.m.)   
108   10.55         house    2700.0      Plot area 300(250.84 sq.m.)   
113    4.50         house    1152.0      Plot area 128(107.02 sq.m.)   
127    5.65         house    2367.0       Plot area 263(219.9 sq.m.)   
129    0.59         house     585.0       Plot area 585(54.35 sq.m.)   
131    3.50         house    1080.0      Plot area 120(100.34 sq.m.)   
134     NaN         house       NaN      Plot area 520(434.79 sq.m.)   
141    0.65         house     720.0        Plot area 80(66.89 sq.m.)   
143    8.40         house    2430.0      Plot area 270(225.75 sq.m.)   
162    0.55         house    1350.0     Plot area 1350(125.42 sq.m.)   
198   17.00         house    4518.0      Plot area 502(419.74 sq.m.)   
202    9.00         house    3240.0      Plot area 360(301.01 sq.m.)   
208    7.00         house    5400.0      Plot area 600(501.68 sq.m.)   
211    4.75         house    2340.0      Plot area 260(217.39 sq.m.)   
217    5.80         house    1800.0      Plot area 200(167.23 sq.m.)   
233    3.60         house    1449.0      Plot area 161(134.62 sq.m.)   
253    0.70         house     684.0       Plot area 684(63.55 sq.m.)   
262    7.25         house    2475.0      Plot area 275(229.94 sq.m.)   
264    4.70         house    2610.0      Plot area 290(242.48 sq.m.)   
267    3.65         house    3450.0     Plot area 3450(320.52 sq.m.)   
268    5.00         house    2367.0       Plot area 263(219.9 sq.m.)   
269    5.00         house    2250.0      Plot area 250(209.03 sq.m.)   
274    5.00         house    3078.0      Plot area 342(285.96 sq.m.)   
281    1.50         house    1980.0      Plot area 220(183.95 sq.m.)   
283    9.40         house    3600.0      Plot area 400(334.45 sq.m.)   
291   25.00         house    8036.0     Plot area 1000(836.13 sq.m.)   
295   23.00         house    9000.0     Plot area 1000(836.13 sq.m.)   
316    3.30         house    2250.0      Plot area 250(209.03 sq.m.)   
320   15.00         house    4500.0      Plot area 500(418.06 sq.m.)   
324    1.60         house     558.0        Plot area 62(51.84 sq.m.)   
332    6.50         house    1800.0      Plot area 200(167.23 sq.m.)   
339   18.90         house    4518.0      Plot area 502(419.74 sq.m.)   
344    7.25         house    2700.0      Plot area 300(250.84 sq.m.)   
345    9.00         house     900.0       Plot area 900(83.61 sq.m.)   
364    4.65         house    2646.0      Plot area 294(245.82 sq.m.)   
379    2.20         house    3200.0     Plot area 3200(297.29 sq.m.)   
409    6.80         house    3240.0      Plot area 360(301.01 sq.m.)   
414    1.50         house    1300.0     Plot area 1300(120.77 sq.m.)   
422    0.60         house      57.0          Plot area 57(5.3 sq.m.)   
428    3.40         house    2250.0      Plot area 250(209.03 sq.m.)   
442    0.92         house     603.0        Plot area 67(56.02 sq.m.)   
445   12.00         house    3240.0      Plot 

In [ ]:
def convert_scale(row):
    if np.isnan(row['area']) or np.isnan(row['built_up_area']):
        return row['built_up_area']
    else:
        if round(row['area']/row['built_up_area']) == 9.0:
            return row['built_up_area'] * 9
        elif round(row['area']/row['built_up_area']) == 11.0:
            return row['built_up_area'] * 10.7
        else:
            return row['built_up_area']

In [ ]:
all_nan_df['built_up_area'] = all_nan_df.apply(convert_scale,axis=1)

In [ ]:
all_nan_df

price property_type      area                     areaWithType  \
4      2.00         house    1337.0   Plot area 1337.4(124.25 sq.m.)   
11     0.70         house     864.0       Plot area 864(80.27 sq.m.)   
22     8.49         house    2700.0      Plot area 300(250.84 sq.m.)   
39     2.25         house     900.0       Plot area 100(83.61 sq.m.)   
45    13.50         house    4518.0      Plot area 502(419.74 sq.m.)   
52    12.00         house    3150.0      Plot area 350(292.64 sq.m.)   
55     7.80         house    3370.0      Plot area 360(301.01 sq.m.)   
56    10.50         house    3240.0      Plot area 360(301.01 sq.m.)   
57    17.00         house    4500.0     Plot area 4500(418.06 sq.m.)   
70     0.27         house     225.0        Plot area 225(20.9 sq.m.)   
71     4.12         house    4635.0      Plot area 515(430.61 sq.m.)   
93     0.53         house     363.0       Plot area 360(33.45 sq.m.)   
94     0.82         house     900.0       Plot area 100(83.61 sq.m.)   
98     0.50         house     450.0       Plot area 450(41.81 sq.m.)   
108   10.55         house    2700.0      Plot area 300(250.84 sq.m.)   
113    4.50         house    1152.0      Plot area 128(107.02 sq.m.)   
127    5.65         house    2367.0       Plot area 263(219.9 sq.m.)   
129    0.59         house     585.0       Plot area 585(54.35 sq.m.)   
131    3.50         house    1080.0      Plot area 120(100.34 sq.m.)   
134     NaN         house       NaN      Plot area 520(434.79 sq.m.)   
141    0.65         house     720.0        Plot area 80(66.89 sq.m.)   
143    8.40         house    2430.0      Plot area 270(225.75 sq.m.)   
162    0.55         house    1350.0     Plot area 1350(125.42 sq.m.)   
198   17.00         house    4518.0      Plot area 502(419.74 sq.m.)   
202    9.00         house    3240.0      Plot area 360(301.01 sq.m.)   
208    7.00         house    5400.0      Plot area 600(501.68 sq.m.)   
211    4.75         house    2340.0      Plot area 260(217.39 sq.m.)   
217    5.80         house    1800.0      Plot area 200(167.23 sq.m.)   
233    3.60         house    1449.0      Plot area 161(134.62 sq.m.)   
253    0.70         house     684.0       Plot area 684(63.55 sq.m.)   
262    7.25         house    2475.0      Plot area 275(229.94 sq.m.)   
264    4.70         house    2610.0      Plot area 290(242.48 sq.m.)   
267    3.65         house    3450.0     Plot area 3450(320.52 sq.m.)   
268    5.00         house    2367.0       Plot area 263(219.9 sq.m.)   
269    5.00         house    2250.0      Plot area 250(209.03 sq.m.)   
274    5.00         house    3078.0      Plot area 342(285.96 sq.m.)   
281    1.50         house    1980.0      Plot area 220(183.95 sq.m.)   
283    9.40         house    3600.0      Plot area 400(334.45 sq.m.)   
291   25.00         house    8036.0     Plot area 1000(836.13 sq.m.)   
295   23.00         house    9000.0     Plot area 1000(836.13 sq.m.)   
316    3.30         house    2250.0      Plot area 250(209.03 sq.m.)   
320   15.00         house    4500.0      Plot area 500(418.06 sq.m.)   
324    1.60         house     558.0        Plot area 62(51.84 sq.m.)   
332    6.50         house    1800.0      Plot area 200(167.23 sq.m.)   
339   18.90         house    4518.0      Plot area 502(419.74 sq.m.)   
344    7.25         house    2700.0      Plot area 300(250.84 sq.m.)   
345    9.00         house     900.0       Plot area 900(83.61 sq.m.)   
364    4.65         house    2646.0      Plot area 294(245.82 sq.m.)   
379    2.20         house    3200.0     Plot area 3200(297.29 sq.m.)   
409    6.80         house    3240.0      Plot area 360(301.01 sq.m.)   
414    1.50         house    1300.0     Plot area 1300(120.77 sq.m.)   
422    0.60         house      57.0          Plot area 57(5.3 sq.m.)   
428    3.40         house    2250.0      Plot area 250(209.03 sq.m.)   
442    0.92         house     603.0        Plot area 67(56.02 sq.m.)   
445   12.00         house    3240.0      Plot 

In [ ]:
# update the original dataframe
df.update(all_nan_df)

In [ ]:
df.isnull().sum()

property_type             0
society                   1
sector                    0
price                    18
price_per_sqft           18
area                     18
areaWithType              0
bedRoom                   0
bathroom                  0
balcony                   0
additionalRoom            0
floorNum                 19
facing                 1105
agePossession             1
nearbyLocations         177
furnishDetails          981
features                635
super_built_up_area    1888
built_up_area          2070
carpet_area            1859
dtype: int64

In [ ]:
df.head(1)

property_type      society     sector  price  price_per_sqft    area  \
0         house  independent  sector 81   0.89          8127.0  1095.0   

                         areaWithType  bedRoom  bathroom balcony  \
0  Built Up area: 1095 (101.73 sq.m.)        1         1       0   

  additionalRoom  floorNum facing agePossession  \
0  not available       1.0    NaN     undefined   

                                                                                                                                                                                                                           nearbyLocations  \
0  ['Orris Community Center', 'Petrol Pump', 'HP Petrol Pump', 'Essar Petrol Pump', 'Vijay Petrol Pump', 'Petrol Pump Indian Oil', 'Petrol pump Maitri motors', 'Indian Oil', "McDonald's", 'Rao Dhaba', 'KFC', 'Pizza Hut', "McDonald's"]   

                                                                                                                                                                                                                                                         furnishDetails  \
0  ['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']   

  features  super_built_up_area  built_up_area  carpet_area  
0      NaN                  NaN         1095.0          NaN

**2. additionalRoom**

In [ ]:
df['additionalRoom'].value_counts()

not available                                    1587
servant room                                      705
study room                                        250
others                                            225
pooja room                                        165
study room,servant room                            99
store room                                         99
pooja room,servant room                            82
pooja room,study room,servant room,store room      72
servant room,others                                60
pooja room,study room,servant room                 55
pooja room,study room,servant room,others          54
servant room,pooja room                            38
servant room,store room                            33
study room,others                                  29
pooja room,study room                              22
pooja room,others                                  17
pooja room,store room                              15
pooja room,store room,study 

In [ ]:
# additional room
# List of new columns to be created
new_cols = ['study room', 'servant room', 'store room', 'pooja room', 'others']

# Populate the new columns based on the "additionalRoom" column
for col in new_cols:
    df[col] = df['additionalRoom'].str.contains(col).astype(int)

In [ ]:
df.sample(5)[['additionalRoom','study room', 'servant room', 'store room', 'pooja room', 'others']]

additionalRoom  study room  servant room  store room  pooja room  others
1104     pooja room           0             0           0           1       0
840      study room           1             0           0           0       0
2620   servant room           0             1           0           0       0
1326   servant room           0             1           0           0       0
827    servant room           0             1           0           0       0

In [ ]:
df.head()

property_type                           society     sector  price  \
0         house                       independent  sector 81   0.89   
1          flat        central park flower valley  sector 33   1.35   
2          flat            guru gram haryana cghs  sector 56   1.39   
3          flat  emaar mgf emerald floors premier  sector 65   3.00   
4         house                       independent    manesar   2.00   

   price_per_sqft    area  \
0          8127.0  1095.0   
1         10384.0  1300.0   
2          8424.0  1650.0   
3         15189.0  1975.0   
4         14954.0  1337.0   

                                                                                                             areaWithType  \
0                                                                                      Built Up area: 1095 (101.73 sq.m.)   
1                                                                                        Carpet area: 1300 (120.77 sq.m.)   
2  Super Built up area 1850(171.87 sq.m.)Built Up area: 1750 sq.ft. (162.58 sq.m.)Carpet area: 1650 sq.ft. (153.29 sq.m.)   
3                                                                                  Super Built up area 1975(183.48 sq.m.)   
4                                                                                          Plot area 1337.4(124.25 sq.m.)   

   bedRoom  bathroom balcony           additionalRoom  floorNum      facing  \
0        1         1       0            not available       1.0         NaN   
1        3         2       2            not available       3.0        West   
2        3         2       3  pooja room,servant room       6.0       South   
3        4         4      3+             servant room       2.0  South-East   
4        3         1       0            not available       3.0       North   

      agePossession  \
0         undefined   
1   0 to 1 Year Old   
2      10+ Year Old   
3   1 to 5 Year Old   
4  5 to 10 Year Old   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           nearbyLocations  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ['Orris Community Center', 'Petrol Pump', 'HP Petrol Pump', 'Essar Petrol Pump', 'Vijay Petrol Pump', 'Petrol Pump Indian Oil', 'Petrol pump Maitri motors', 'Indian Oil', "McDonald's", 'Rao Dhaba', 'KFC', 'Pizza Hut', "McDonald's"]   
1                                                                                                                                                                                                                      

**3.agePossession**

In [ ]:
df['agePossession'].value_counts()

1 to 5 Year Old       1676
5 to 10 Year Old       575
0 to 1 Year Old        530
undefined              332
10+ Year Old           310
Under Construction      90
Within 6 months         70
Within 3 months         26
Dec-23                  22
By 2023                 19
By 2024                 17
Dec-24                  16
Mar-24                  14
Jan-24                   8
Oct-24                   8
Aug-23                   7
Jun-24                   7
Dec-25                   7
Nov-23                   5
Sep-23                   4
Jul-24                   4
By 2025                  4
Oct-23                   4
Aug-24                   4
May-24                   3
Jan-25                   3
Feb-24                   3
Nov-24                   3
Jan-26                   2
Aug-25                   2
Mar-25                   2
Jun-27                   2
Jul-25                   2
Jul-27                   2
By 2027                  2
Dec-26                   2
Oct-25                   2
S

In [ ]:
def categorize_age_possession(value):
    if pd.isna(value):
        return "Undefined"
    if "0 to 1 Year Old" in value or "Within 6 months" in value or "Within 3 months" in value:
        return "New Property"
    if "1 to 5 Year Old" in value:
        return "Relatively New"
    if "5 to 10 Year Old" in value:
        return "Moderately Old"
    if "10+ Year Old" in value:
        return "Old Property"
    if "Under Construction" in value or "By" in value:
        return "Under Construction"
    try:
        # For entries like 'May 2024'
        int(value.split(" ")[-1])
        return "Under Construction"
    except:
        return "Undefined"

In [ ]:
df['agePossession'] = df['agePossession'].apply(categorize_age_possession)

In [ ]:
df['agePossession'].value_counts()

Relatively New        1676
New Property           626
Moderately Old         575
Undefined              484
Old Property           310
Under Construction     132
Name: agePossession, dtype: int64

In [ ]:
df.head()

property_type                           society     sector  price  \
0         house                       independent  sector 81   0.89   
1          flat        central park flower valley  sector 33   1.35   
2          flat            guru gram haryana cghs  sector 56   1.39   
3          flat  emaar mgf emerald floors premier  sector 65   3.00   
4         house                       independent    manesar   2.00   

   price_per_sqft    area  \
0          8127.0  1095.0   
1         10384.0  1300.0   
2          8424.0  1650.0   
3         15189.0  1975.0   
4         14954.0  1337.0   

                                                                                                             areaWithType  \
0                                                                                      Built Up area: 1095 (101.73 sq.m.)   
1                                                                                        Carpet area: 1300 (120.77 sq.m.)   
2  Super Built up area 1850(171.87 sq.m.)Built Up area: 1750 sq.ft. (162.58 sq.m.)Carpet area: 1650 sq.ft. (153.29 sq.m.)   
3                                                                                  Super Built up area 1975(183.48 sq.m.)   
4                                                                                          Plot area 1337.4(124.25 sq.m.)   

   bedRoom  bathroom balcony           additionalRoom  floorNum      facing  \
0        1         1       0            not available       1.0         NaN   
1        3         2       2            not available       3.0        West   
2        3         2       3  pooja room,servant room       6.0       South   
3        4         4      3+             servant room       2.0  South-East   
4        3         1       0            not available       3.0       North   

    agePossession  \
0       Undefined   
1    New Property   
2    Old Property   
3  Relatively New   
4  Moderately Old   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           nearbyLocations  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ['Orris Community Center', 'Petrol Pump', 'HP Petrol Pump', 'Essar Petrol Pump', 'Vijay Petrol Pump', 'Petrol Pump Indian Oil', 'Petrol pump Maitri motors', 'Indian Oil', "McDonald's", 'Rao Dhaba', 'KFC', 'Pizza Hut', "McDonald's"]   
1                                                                                                                                                                                                                                  

**4.furnishDetails**

In [ ]:
df.sample(5)[['furnishDetails','features']]

furnishDetails  \
3442  ['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']   
1414    ['7 Wardrobe', '17 Light', '5 AC', '1 Modular Kitchen', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']   
1298                                                                                                                                                                                                                                                                    []   
2696  ['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']   
1295  ['3 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                       features  
3442                                                                                                                                                                                                                                                                                                                                                                                                                                                        NaN  
1414                                                                                                                                                                                                                                                  ['Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Swimming Pool', 'Park', 'Security Personnel', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center', 'Rain Water Harvesting']  
1298                                                                                                                                                                                                                                                                                                                                                                                                                                                        NaN  
2696                                                                                                                                                                                                                                                                                                                                                                                                                                                        NaN  
1295  ['Security / Fire Alarm', 'Feng Shui / Vaastu Compliant', 'Intercom Facility', 'Lift(s)', 'High Ceiling Height', 'Maintenance Staff', 'Water Storage', 'Separate entry for servant room', 'No open drainage around', 'Visitor Parking', 'Swimming Pool', 'Park', 'Security Personnel', 'Natural Light', 'Internet/wi-fi connectivity', 'Airy Rooms', 'Shopping Centre', 'Fitness Centre

In [ ]:
# Extract all unique furnishings from the furnishDetails column
all_furnishings = []
for detail in df['furnishDetails'].dropna():
    furnishings = detail.replace('[', '').replace(']', '').replace("'", "").split(', ')
    all_furnishings.extend(furnishings)
unique_furnishings = list(set(all_furnishings))

# Define a function to extract the count of a furnishing from the furnishDetails
def get_furnishing_count(details, furnishing):
    if isinstance(details, str):
        if f"No {furnishing}" in details:
            return 0
        pattern = re.compile(f"(\d+) {furnishing}")
        match = pattern.search(details)
        if match:
            return int(match.group(1))
        elif furnishing in details:
            return 1
    return 0

# Simplify the furnishings list by removing "No" prefix and numbers
columns_to_include = [re.sub(r'No |\d+', '', furnishing).strip() for furnishing in unique_furnishings]
columns_to_include = list(set(columns_to_include))  # Get unique furnishings
columns_to_include = [furnishing for furnishing in columns_to_include if furnishing]  # Remove empty strings

# Create new columns for each unique furnishing and populate with counts
for furnishing in columns_to_include:
    df[furnishing] = df['furnishDetails'].apply(lambda x: get_furnishing_count(x, furnishing))

# Create the new dataframe with the required columns
furnishings_df = df[['furnishDetails'] + columns_to_include]

In [ ]:
furnishings_df.shape

(3803, 19)

In [ ]:
furnishings_df.drop(columns=['furnishDetails'],inplace=True)

<ipython-input-56-e4dadedf8e21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  furnishings_df.drop(columns=['furnishDetails'],inplace=True)


In [ ]:
furnishings_df.sample(5)

Geyser  Sofa  Microwave  Bed  Exhaust Fan  Fridge  Curtains  \
1732       0     0          0    0            0       0         0   
868        0     0          0    0            0       0         0   
2559       0     0          0    0            0       0         0   
1409       0     0          0    0            0       0         0   
2514       0     0          0    0            0       0         0   

      Water Purifier  Dining Table  TV  Chimney  AC  Light  Washing Machine  \
1732               0             0   0        0   0      0                0   
868                0             0   0        0   0      1                0   
2559               0             0   0        0   0      0                0   
1409               0             0   0        0   0      0                0   
2514               0             0   0        0   0      0                0   

      Wardrobe  Stove  Modular Kitchen  Fan  
1732         0      0                0    0  
868          1      0                0    1  
2559         0      0                0    0  
1409         0      0                0    0  
2514         0      0                0    0

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(furnishings_df)

In [ ]:
wcss_reduced = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(scaled_data)
    wcss_reduced.append(kmeans.inertia_)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [ ]:
# Plot the results
plt.figure(figsize=(12, 8))
plt.plot(range(1,11), wcss_reduced, marker='o', linestyle='--')
plt.title('Elbow Method For Optimal Number of Clusters (Reduced Range)')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.grid(True)
plt.show()

In [ ]:
n_clusters = 3

# Fit the KMeans model
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(scaled_data)

# Predict the cluster assignments for each row
cluster_assignments = kmeans.predict(scaled_data)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
df = df.iloc[:,:-18]

In [ ]:
df['furnishing_type'] = cluster_assignments

In [ ]:
df.sample(5)[['furnishDetails','furnishing_type']]
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished

furnishDetails  \
639       ['4 Fan', '1 Exhaust Fan', '3 Geyser', '8 Light', '3 AC', '1 Chimney', '1 Modular Kitchen', '6 Wardrobe', 'No Bed', 'No Curtains', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']   
2137     ['1 Fan', '1 Exhaust Fan', '1 Geyser', '1 Light', '1 Curtains', '1 Modular Kitchen', '1 Wardrobe', 'No AC', 'No Bed', 'No Chimney', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']   
2759                                                                                                                                                                                                                                                                NaN   
3065                                                                                                                                                                                                                                                                NaN   
2643  ['1 Wardrobe', '1 Stove', '1 Modular Kitchen', '1 Chimney', 'No AC', 'No Bed', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Light', 'No Microwave', 'No Fridge', 'No Sofa', 'No TV', 'No Washing Machine', 'No Water Purifier']   

      furnishing_type  
639                 2  
2137                0  
2759                0  
3065                0  
2643                0

**5.features**

In [ ]:
df[['society','features']].sample(5)

society  \
3265  indiabulls centrum park   
392          dlf the skycourt   
2819      new colony, gurgaon   
1860         ansal heights 86   
3241           pivotal devaan   

                                                                                                                                                                                                                                                                                            features  
3265                                                                                                                                      ['Power Back-up', 'Lift(s)', 'Swimming Pool', 'Park', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center', 'Rain Water Harvesting']  
392            ['Security / Fire Alarm', 'Power Back-up', 'Feng Shui / Vaastu Compliant', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Piped-gas', 'Visitor Parking', 'Swimming Pool', 'Park', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center']  
2819                                                                ['Security / Fire Alarm', 'Lift(s)', 'High Ceiling Height', 'Maintenance Staff', 'False Ceiling Lighting', 'Water Storage', 'Park', 'Recently Renovated', 'Security Personnel', 'Natural Light', 'Airy Rooms', 'Waste Disposal']  
1860                                                                                                                                                                                               ['Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Visitor Parking']  
3241  ['Security / Fire Alarm', 'Power Back-up', 'Feng Shui / Vaastu Compliant', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Visitor Parking', 'Swimming Pool', 'Park', 'Security Personnel', 'Waste Disposal', 'Rain Water Harvesting', 'Club house / Community Center']

In [ ]:
df['features'].isnull().sum()

635

In [ ]:
import pandas as pd
app_df = pd.read_csv('appartments.csv')
app_df.head(2)

PropertyName                               PropertySubName  \
0  Smartworld One DXP  2, 3, 4 BHK Apartment in Sector 113, Gurgaon   
1           M3M Crown     3, 4 BHK Apartment in Sector 111, Gurgaon   

                                                                                                                                  NearbyLocations  \
0                                           ['Bajghera Road', 'Palam Vihar Halt', 'DPSG Palam Vihar', 'Park Hospital', 'Gurgaon Railway Station']   
1  ['DPSG Palam Vihar Gurugram', 'The NorthCap University', 'Park Hospital, Palam Vihar', 'Pacific D21 Mall', 'Palam Vihar Halt Railway Station']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    LocationAdvantages  \
0         {'Bajghera Road': '800 Meter', 'Palam Vihar Halt': '2.5 KM', 'DPSG Palam Vihar': '3.1 KM', 'Park Hospital': '3.1 KM', 'Gurgaon Railway Station': '4.9 KM', 'The NorthCap University': '5.4 KM', 'Dwarka Expy': '1.2 KM', 'Hyatt Place Gurgaon Udyog Vihar': '7.7 KM', 'Dwarka Sector 21, Metro Station': '7.2 KM', 'Pacific D21 Mall': '7.4 KM', 'Indira Gandhi International Airport': '14.7 KM', 'Hamoni Golf Camp': '6.2 KM', 'Fun N Food Waterpark': '8.8 KM', 'Accenture DDC5': '9 KM'}   
1  {'DPSG Palam Vihar Gurugram': '1.4 Km', 'The NorthCap University': '4.4 Km', 'Park Hospital, Palam Vihar': '1.4 Km', 'Pacific D21 Mall': '8.2 Km', 'Palam Vihar Halt Railway Station': '1.2 Km', 'Dwarka Sector 21 Metro Station': '8.1 Km', 'Dwarka Expressway': '450 m', 'Fun N Food Water Park': '8.1 Km', 'Indira Gandhi International Airport': '14.1 Km', 'Tau DeviLal Sports Complex': '11.2 Km', 'Hamoni Golf Camp': '5 Km', 'Hyatt Place': '6.1 Km', 'Altrade Business Centre': '11.2 Km'}   

                                                                          Link  \
0  https://www.99acres.com/smartworld-one-dxp-sector-113-gurgaon-npxid-r400415   
1           https://www.99acres.com/m3m-crown-sector-111-gurgaon-npxid-r404068   

                                                                                                                                                                                                                                                                                                                                                                                           PriceDetails  \
0  {'2 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,370 sq.ft.', 'price-range': '₹ 2 - 2.4 Cr'}, '3 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,850 - 2,050 sq.ft.', 'price-range': '₹ 2.25 - 3.59 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,600 sq.ft.', 'price-range': '₹ 3.24 - 4.56 Cr'}}   
1                                                                                                       {'3 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '1,605 - 2,170 sq.ft.', 'price-range': '₹ 2.2 - 3.03 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '2,248 - 2,670 sq.ft.', 'price-range': '₹ 3.08 - 3.73 Cr'}}   

                                                                                                                                   TopFacilities  
0                     ['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']  
1  ['Bowling Alley', 'Mini Theatre', 'Manicured Garden', 'Swimming Pool', 'Flower Garden', 'Reading Lounge', 'Golf Course', 'Barbecue', 'Sauna']

In [ ]:
app_df['PropertyName'] = app_df['PropertyName'].str.lower()

In [ ]:
temp_df = df[df['features'].isnull()]

In [ ]:
temp_df.shape

(635, 26)

In [ ]:
x = temp_df.merge(app_df,left_on='society',right_on='PropertyName',how='left')['TopFacilities']

In [ ]:
df.loc[temp_df.index,'features'] = x.values

In [ ]:
df['features'].isnull().sum()

481

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import ast

In [ ]:
# Convert the string representation of lists in the 'features' column to actual lists
df['features_list'] = df['features'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) and x.startswith('[') else [])

# Use MultiLabelBinarizer to convert the features list into a binary matrix
mlb = MultiLabelBinarizer()
features_binary_matrix = mlb.fit_transform(df['features_list'])

# Convert the binary matrix into a DataFrame
features_binary_df = pd.DataFrame(features_binary_matrix, columns=mlb.classes_)

In [ ]:
features_binary_df.sample(5)

24/7 Power Backup  24/7 Water Supply  24x7 Security  ATM  \
1260                  0                  0              0    0   
273                   0                  0              0    0   
2386                  0                  0              0    0   
3498                  0                  0              0    0   
1283                  0                  0              0    0   

      Aerobics Centre  Air Hockey  Airy Rooms  Amphitheatre  \
1260                0           0           0             0   
273                 0           0           0             0   
2386                0           0           0             0   
3498                0           0           1             0   
1283                0           0           1             0   

      Automated Car Wash  Badminton Court  Bank Attached Property  \
1260                   0                0                       0   
273                    0                0                       0   
2386                   0                0                       0   
3498                   0                0                       1   
1283                   0                0                       1   

      Banquet Hall  Bar/Chill-Out Lounge  Barbecue  Basketball Court  \
1260             0                     0         0                 0   
273              0                     0         0                 0   
2386             0                     0         0                 0   
3498             0                     0         0                 0   
1283             0                     0         0                 0   

      Beach Volley Ball Court  Billiards  Bowling Alley  Bus Shelter  \
1260                        0          0              0            0   
273                         0          0              0            0   
2386                        0          0              0            0   
3498                        0          0              0            0   
1283                        0          0              0            0   

      Business Lounge  CCTV Camera Security  Cafeteria  Car Parking  \
1260                0                     0          0            0   
273                 0                     0          0            0   
2386                0                     0          0            0   
3498                0                     0          0            0   
1283                0                     0          0            0   

      Car wash area  Card Room  Centrally Air Conditioned  Changing Area  \
1260              0          0                          0              0   
273               0          1                          0              0   
2386              0          0                          0              0   
3498              0          0                          0              0   
1283              0          0                          1              0   

      Children's Play Area  Cigar Lounge  Clinic  Club House  \
1260                     0             0       0           0   
273                      0             0       0           0   
2386                     0             0       0           0   
3498                     0             0       0           0   
1283                     0             0       0           0   

      Club house / Community Center  Community Hall  Concierge Service  \
1260                              0               0                  0   
273                               0               0                  0   
2386                              0               0                  0   
3498                              1               0                  0   
1283                              1               0                  0   

      Conference room  Creche/Day care  Cricket Pitch  Doctor on Call  \
1260                0                0              0               0   
273                 1                0              0               0   
2386            

In [ ]:
features_binary_df.shape

(3803, 130)

In [ ]:
wcss_reduced = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(features_binary_df)
    wcss_reduced.append(kmeans.inertia_)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [ ]:
# Plot the results
plt.figure(figsize=(12, 8))
plt.plot(range(1,11), wcss_reduced, marker='o', linestyle='--')
plt.title('Elbow Method For Optimal Number of Clusters (Reduced Range)')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.grid(True)
plt.show()

In [ ]:
# Define the weights for each feature as provided
# Assigning weights based on perceived luxury contribution
weights = {
    '24/7 Power Backup': 8,
    '24/7 Water Supply': 4,
    '24x7 Security': 7,
    'ATM': 4,
    'Aerobics Centre': 6,
    'Airy Rooms': 8,
    'Amphitheatre': 7,
    'Badminton Court': 7,
    'Banquet Hall': 8,
    'Bar/Chill-Out Lounge': 9,
    'Barbecue': 7,
    'Basketball Court': 7,
    'Billiards': 7,
    'Bowling Alley': 8,
    'Business Lounge': 9,
    'CCTV Camera Security': 8,
    'Cafeteria': 6,
    'Car Parking': 6,
    'Card Room': 6,
    'Centrally Air Conditioned': 9,
    'Changing Area': 6,
    "Children's Play Area": 7,
    'Cigar Lounge': 9,
    'Clinic': 5,
    'Club House': 9,
    'Concierge Service': 9,
    'Conference room': 8,
    'Creche/Day care': 7,
    'Cricket Pitch': 7,
    'Doctor on Call': 6,
    'Earthquake Resistant': 5,
    'Entrance Lobby': 7,
    'False Ceiling Lighting': 6,
    'Feng Shui / Vaastu Compliant': 5,
    'Fire Fighting Systems': 8,
    'Fitness Centre / GYM': 8,
    'Flower Garden': 7,
    'Food Court': 6,
    'Foosball': 5,
    'Football': 7,
    'Fountain': 7,
    'Gated Community': 7,
    'Golf Course': 10,
    'Grocery Shop': 6,
    'Gymnasium': 8,
    'High Ceiling Height': 8,
    'High Speed Elevators': 8,
    'Infinity Pool': 9,
    'Intercom Facility': 7,
    'Internal Street Lights': 6,
    'Internet/wi-fi connectivity': 7,
    'Jacuzzi': 9,
    'Jogging Track': 7,
    'Landscape Garden': 8,
    'Laundry': 6,
    'Lawn Tennis Court': 8,
    'Library': 8,
    'Lounge': 8,
    'Low Density Society': 7,
    'Maintenance Staff': 6,
    'Manicured Garden': 7,
    'Medical Centre': 5,
    'Milk Booth': 4,
    'Mini Theatre': 9,
    'Multipurpose Court': 7,
    'Multipurpose Hall': 7,
    'Natural Light': 8,
    'Natural Pond': 7,
    'Park': 8,
    'Party Lawn': 8,
    'Piped Gas': 7,
    'Pool Table': 7,
    'Power Back up Lift': 8,
    'Private Garden / Terrace': 9,
    'Property Staff': 7,
    'RO System': 7,
    'Rain Water Harvesting': 7,
    'Reading Lounge': 8,
    'Restaurant': 8,
    'Salon': 8,
    'Sauna': 9,
    'Security / Fire Alarm': 9,
    'Security Personnel': 9,
    'Separate entry for servant room': 8,
    'Sewage Treatment Plant': 6,
    'Shopping Centre': 7,
    'Skating Rink': 7,
    'Solar Lighting': 6,
    'Solar Water Heating': 7,
    'Spa': 9,
    'Spacious Interiors': 9,
    'Squash Court': 8,
    'Steam Room': 9,
    'Sun Deck': 8,
    'Swimming Pool': 8,
    'Temple': 5,
    'Theatre': 9,
    'Toddler Pool': 7,
    'Valet Parking': 9,
    'Video Door Security': 9,
    'Visitor Parking': 7,
    'Water Softener Plant': 7,
    'Water Storage': 7,
    'Water purifier': 7,
    'Yoga/Meditation Area': 7
}
# Calculate luxury score for each row
luxury_score = features_binary_df[list(weights.keys())].multiply(list(weights.values())).sum(axis=1)

In [ ]:
df['luxury_score'] = luxury_score

In [ ]:
df.head()

property_type                           society     sector  price  \
0         house                       independent  sector 81   0.89   
1          flat        central park flower valley  sector 33   1.35   
2          flat            guru gram haryana cghs  sector 56   1.39   
3          flat  emaar mgf emerald floors premier  sector 65   3.00   
4         house                       independent    manesar   2.00   

   price_per_sqft    area  \
0          8127.0  1095.0   
1         10384.0  1300.0   
2          8424.0  1650.0   
3         15189.0  1975.0   
4         14954.0  1337.0   

                                                                                                             areaWithType  \
0                                                                                      Built Up area: 1095 (101.73 sq.m.)   
1                                                                                        Carpet area: 1300 (120.77 sq.m.)   
2  Super Built up area 1850(171.87 sq.m.)Built Up area: 1750 sq.ft. (162.58 sq.m.)Carpet area: 1650 sq.ft. (153.29 sq.m.)   
3                                                                                  Super Built up area 1975(183.48 sq.m.)   
4                                                                                          Plot area 1337.4(124.25 sq.m.)   

   bedRoom  bathroom balcony           additionalRoom  floorNum      facing  \
0        1         1       0            not available       1.0         NaN   
1        3         2       2            not available       3.0        West   
2        3         2       3  pooja room,servant room       6.0       South   
3        4         4      3+             servant room       2.0  South-East   
4        3         1       0            not available       3.0       North   

    agePossession  \
0       Undefined   
1    New Property   
2    Old Property   
3  Relatively New   
4  Moderately Old   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           nearbyLocations  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ['Orris Community Center', 'Petrol Pump', 'HP Petrol Pump', 'Essar Petrol Pump', 'Vijay Petrol Pump', 'Petrol Pump Indian Oil', 'Petrol pump Maitri motors', 'Indian Oil', "McDonald's", 'Rao Dhaba', 'KFC', 'Pizza Hut', "McDonald's"]   
1                                                                                                                                                                                                                                  

In [ ]:
# cols to drop -> nearbyLocations,furnishDetails, features,features_list, additionalRoom
df.drop(columns=['nearbyLocations','furnishDetails','features','features_list','additionalRoom'],inplace=True)

In [ ]:
df.sample(5)

property_type                           society      sector  price  \
2618          flat            vatika the seven lamps   sector 82   0.89   
1717          flat                       birla navya  sector 63a   3.70   
199           flat                   the close north   sector 50   2.75   
1987          flat  emaar mgf emerald floors premier   sector 65   2.24   
3012          flat                ireo the corridors  sector 67a   1.80   

      price_per_sqft    area  \
2618          8850.0  1006.0   
1717         15416.0  2400.0   
199          12500.0  2200.0   
1987         13575.0  1650.0   
3012         14658.0  1228.0   

                                                                                                                areaWithType  \
2618                                          Super Built up area 1428.08(132.67 sq.m.)Carpet area: 1000 sq.ft. (92.9 sq.m.)   
1717                                                                                        Carpet area: 2400 (222.97 sq.m.)   
199   Super Built up area 2605(242.01 sq.m.)Built Up area: 2400 sq.ft. (222.97 sq.m.)Carpet area: 2200 sq.ft. (204.39 sq.m.)   
1987                                                                                        Carpet area: 1650 (153.29 sq.m.)   
3012                                        Super Built up area 1921(178.47 sq.m.)Carpet area: 1227.96 sq.ft. (114.08 sq.m.)   

      bedRoom  bathroom balcony  floorNum      facing       agePossession  \
2618        2         2       2      14.0        East      Relatively New   
1717        3         3      3+       1.0       North  Under Construction   
199         3         3       3       3.0  South-East      Moderately Old   
1987        3         3       3       2.0  North-West      Relatively New   
3012        3         3       3       6.0       North        New Property   

      super_built_up_area  built_up_area  carpet_area  study room  \
2618              1428.08            NaN      1000.00           1   
1717                  NaN            NaN      2400.00           0   
199               2605.00         2400.0      2200.00           0   
1987                  NaN            NaN      1650.00           1   
3012              1921.00            NaN      1227.96           0   

      servant room  store room  pooja room  others  furnishing_type  \
2618             0           0           0       0                2   
1717             0           0           0       0                0   
199              1           0           0       0                0   
1987             1           0           0       0                0   
3012             1           1           0       0                2   

      luxury_score  
2618           129  
1717            72  
199            110  
1987            46  
3012           150

In [ ]:
df.shape

(3803, 23)

In [ ]:
df.to_csv('properties_cleaned_v2.csv',index=False)